In [2]:
# 다항 로지스틱 회귀분석
import pandas as pd
iris = pd.read_csv('data/iris.csv')
X = iris.drop(['target'], axis = 1)
y = iris['target']

In [4]:
# 훈련용 평가셋 분리하기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, train_size = 0.7, test_size = 0.3, random_state = 123)

In [5]:
# 모델 적합하기
# sklearn l2 패널티를 이용하여, 다중공선성 문제를 내부적으로 해결
# 독립변수 간의 상관성이 높이 변수라면, l2패널티를 0으로 가깝게 하여, 변수를 삭제하는 효과
# 그러나 전통적 통계분석 처럼, p_value 값을 활용하여, 변수들이 유의한지 알아보기는 어렵다.
# 해석력이 중요하면 통계모델, 결과치나 정확도가 중요하면 머신러닝 모델을 사용한다.
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
print(model)

LogisticRegression()


In [11]:
# 모델 평가
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
# 테스트 셋 예측
predicted = model.predict(X_test) # 예측한 확률값을 바탕으로 종속변수 내 unique_value로 리턴
# predicted = model.predict_proba(X_test) # 확률값으로 예측

# 오분류표 생성
cm = confusion_matrix(y_test, predicted)
cmtb = pd.DataFrame(cm, columns = ['predicted_setosa', 'predicted_versicolor', 'predicted_virginica'],
                    index = ['setosa', 'versicolor', 'virginica'])

cmtb

,predicted_setosa,predicted_versicolor,predicted_virginica
setosa,15,0,0
versicolor,0,14,1
virginica,0,0,15


In [14]:
# 정확도
print('Accuracy score : ', accuracy_score(y_test, predicted))

# 분류리포트 생성하기
class_report = classification_report(y_test, predicted)
print(class_report)

Accuracy score :  0.9777777777777777
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        15
Iris-versicolor       1.00      0.93      0.97        15
 Iris-virginica       0.94      1.00      0.97        15

       accuracy                           0.98        45
      macro avg       0.98      0.98      0.98        45
   weighted avg       0.98      0.98      0.98        45



In [15]:
roc_auc_score(y_test, model.predict_proba(X_test), multi_class = 'ovr')

0.9985185185185186

In [16]:
# 다항 로지스틱 회귀 계수 해석

In [17]:
print('Intercept : \n', model.intercept_)
print('Coefficient : \n', model.coef_)

Intercept : 
 [  9.42963576   2.10066476 -11.53030052]
Coefficient : 
 [[-0.45748411  0.87255203 -2.3084507  -0.96016685]
 [ 0.37568618 -0.19450085 -0.16285544 -0.75327619]
 [ 0.08179793 -0.67805118  2.47130614  1.71344303]]


In [18]:
# 오즈비 계산하기
import numpy as np
np.exp(model.coef_)

array([[ 0.63287388,  2.3930101 ,  0.09941516,  0.38282901],
       [ 1.45599014,  0.82324548,  0.84971401,  0.47082152],
       [ 1.0852365 ,  0.50760526, 11.83789871,  5.54803067]])

In [19]:
pd.DataFrame(np.exp(model.coef_), columns = X_train.columns, index = model.classes_)

,sepal length,sepal width,petal length,petal width
Iris-setosa,0.632874,2.393010,0.099415,0.382829
Iris-versicolor,1.455990,0.823245,0.849714,0.470822
Iris-virginica,1.085236,0.507605,11.837899,5.548031


In [20]:
# sepal width가 한 단위 증가할 때, setosa 139% 증가, versicolor 18% 감소, virginica 50% 감소